# SENTIMENT ANALYSIS ON SPEECH

***Gerekli kütüphaneleri import edelim;***

In [1]:
import tscribe
import pandas as pd
import webvtt
import json
import datetime
import time as ptime
from nltk.tokenize import word_tokenize, sent_tokenize

***Konuşmacılar, başlangıç/bitiş değerleri ve konuşmaları***

In [2]:
tscribe.write("C:/Users/10011598/Sentitech/Audio/uzaktan_calisma_performansi_nasil_etkiliyor.json", format="csv", save_as= "C:/Users/10011598/Sentitech/transcript.csv")

C:/Users/10011598/Sentitech/transcript.csv written in 0.28 seconds.


In [3]:
df = pd.read_csv("C:/Users/10011598/Sentitech/transcript.csv");

In [4]:
df.drop(['Unnamed: 0'], axis = 1)

,start_time,end_time,speaker,comment
0,00:00:04,00:00:14,spk_0,"Biz yine bu konuda, iş sonuçlarıyla ilgili han..."
1,00:00:20,00:00:42,spk_1,üretkenlik içinde şunu söyleyebilirim başlığı ...
2,00:00:42,00:01:00,spk_1,"Ee işte iş yapılır, özel hayatın çok fazla ora..."
3,00:01:01,00:01:22,spk_1,Ee dolayısıyla ee hani bu anlamda ben mesela ü...
4,00:01:22,00:01:41,spk_1,şu anda çok daha uykusunu almış bir şekilde ye...
5,00:01:41,00:01:45,spk_1,en basiti kapılarından geçen herkese merhaba d...
6,00:01:45,00:01:46,spk_1,iki
7,00:01:46,00:01:46,spk_2,dakika da olsa
8,00:01:46,00:02:05,spk_1,sohbet etmek istiyor. Yirmi kişi bunu yapsa kı...
9,00:02:06,00:02:19,spk_1,"bir çok çalışır olduk, var yok aslında diyorum..."


***Türkçe köken kontrolü;***

In [5]:
from turkishnlp import detector
obj = detector.TurkishNLP()

In [6]:
obj.is_turkish_origin("program")

False

***Text sınıflandırma işlemleri;***

In [7]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer, AutoModelForSequenceClassification
tokenizer= AutoTokenizer.from_pretrained("savasy/bert-turkish-text-classification")

# build and load model, it take time depending on your internet connection
model= AutoModelForSequenceClassification.from_pretrained("savasy/bert-turkish-text-classification")

# make pipeline
nlp1=pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# apply model
nlp1("bla bla")
# [{'label': 'LABEL_2', 'score': 0.4753005802631378}]

code_to_label={
 'LABEL_0': 'dunya ',
 'LABEL_1': 'ekonomi ',
 'LABEL_2': 'kultur ',
 'LABEL_3': 'saglik ',
 'LABEL_4': 'siyaset ',
 'LABEL_5': 'spor',
 'LABEL_6': 'teknoloji '}

In [8]:
nlp1("Türkiye İsviçre ile oynadığı futbol maçından galip geldi")

[{'label': 'sport', 'score': 0.9447410106658936}]

In [9]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("gurkan08/bert-turkish-text-classification")
model = AutoModelForSequenceClassification.from_pretrained("gurkan08/bert-turkish-text-classification")

nlp2 = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

label_dict = {
 'LABEL_0': 'ekonomi',
 'LABEL_1': 'spor',
 'LABEL_2': 'saglik',
 'LABEL_3': 'kultur_sanat',
 'LABEL_4': 'bilim_teknoloji',
 'LABEL_5': 'egitim'}

In [10]:
nlp2("Türkiye İsviçre ile oynadığı futbol maçından galip geldi")

[{'label': 'spor', 'score': 0.27860233187675476}]

**ÇIKARIM:** *İki farklı model denendiğinde "savasy/bert-turkish-text-classification"in daha başarılı olduğu görülmektedir.*  

***NER;***

In [11]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
model = AutoModelForTokenClassification.from_pretrained("savasy/bert-base-turkish-ner-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-ner-cased")
ner=pipeline('ner', model=model, tokenizer=tokenizer)
ner("Mustafa Kemal Atatürk 19 Mayıs 1919'da Samsun'a ayak bastı.")

[{'word': 'Mustafa',
  'score': 0.9938516616821289,
  'entity': 'B-PER',
  'index': 1,
  'start': 0,
  'end': 7},
 {'word': 'Kemal',
  'score': 0.9881671071052551,
  'entity': 'I-PER',
  'index': 2,
  'start': 8,
  'end': 13},
 {'word': 'Atatürk',
  'score': 0.9957979321479797,
  'entity': 'I-PER',
  'index': 3,
  'start': 14,
  'end': 21},
 {'word': 'Samsun',
  'score': 0.9059982299804688,
  'entity': 'B-LOC',
  'index': 9,
  'start': 39,
  'end': 45}]

In [12]:
ner("Türkiye İsviçre ile oynadığı futbol maçından galip geldi")

[{'word': 'Türkiye',
  'score': 0.8338261246681213,
  'entity': 'B-ORG',
  'index': 1,
  'start': 0,
  'end': 7},
 {'word': 'İsviçre',
  'score': 0.5075637102127075,
  'entity': 'B-LOC',
  'index': 2,
  'start': 8,
  'end': 15}]

***SnowBall Stemmer ile kelimeleri köklerine ayırma;***

In [13]:
from snowballstemmer import TurkishStemmer
turkStem=TurkishStemmer()
turkStem.stemWord("konuda") #konu
turkStem.stemWord("araştırmayla") #araştırma
turkStem.stemWord("sonra") #sonra

'sonra'

In [14]:
turkStem.stemWord("konuda")

'konu'

In [15]:
turkStem.stemWord("araştırmayla")

'araştırma'

In [16]:
turkStem.stemWord("sonra")

'sonra'

***Text olarak alınabilecek kod;***

In [17]:
#!/usr/bin/env python3
def main():
	import sys
	import json
	import datetime
	import codecs

	filename='C:/Users/10011598/Sentitech/Audio/uzaktan_calisma_performansi_nasil_etkiliyor'
	print ("Filename: ", filename+'.json')
	with codecs.open(filename+'.txt', 'w', 'utf-8') as w:
		with codecs.open(filename+'.json', 'r', 'utf-8') as f:
			data=json.loads(f.read())
			labels = data['results']['speaker_labels']['segments']
			speaker_start_times={}
			for label in labels:
				for item in label['items']:
					speaker_start_times[item['start_time']] =item['speaker_label']
			items = data['results']['items']
			lines=[]
			line=''
			time=0
			speaker='null'
			i=0
			for item in items:
				i=i+1
				content = item['alternatives'][0]['content']
				if item.get('start_time'):
					current_speaker=speaker_start_times[item['start_time']]
				elif item['type'] == 'punctuation':
					line = line+content
				if current_speaker != speaker:
					if speaker:
						lines.append({'speaker':speaker, 'line':line, 'time':time})
					line=content
					speaker=current_speaker
					time=item['start_time']
				elif item['type'] != 'punctuation':
					line = line + ' ' + content
			lines.append({'speaker':speaker, 'line':line,'time':time})
			sorted_lines = sorted(lines,key=lambda k: float(k['time']))
          
			for line_data in sorted_lines:
				line='[' + str(datetime.timedelta(seconds=int(round(float(line_data['time']))))) + '] ' + line_data.get('speaker') + ': ' + line_data.get('line')
				w.write(line + '\n\n')


if __name__ == '__main__':
	main()

Filename:  C:/Users/10011598/Sentitech/Audio/uzaktan_calisma_performansi_nasil_etkiliyor.json


In [18]:
main()

Filename:  C:/Users/10011598/Sentitech/Audio/uzaktan_calisma_performansi_nasil_etkiliyor.json


***Diyaloglar için DataFrame olarak alınabilecek kod (1. Yol);***

In [19]:
def speaker():
    import sys
    import json
    import datetime
    import codecs

    filename='C:/Users/10011598/Sentitech/Audio/uzaktan_calisma_performansi_nasil_etkiliyor'
    #print ("Filename: ", filename+'.json')

    with codecs.open(filename+'.json', 'r', 'utf-8') as f:
        data=json.loads(f.read())
        labels = data['results']['speaker_labels']['segments']
        #print(labels)
        speaker_start_times={}
        for label in labels:
            for item in label['items']:
                speaker_start_times[item['start_time']] =item['speaker_label']
        items = data['results']['items']
        line=''
        time=0
        speaker='null'
        i=0
        lines=[]
        time1 = []
        speaker1 = []
        text1 = []
        
        #print(items)
        for item in items:
            i=i+1
            content = item['alternatives'][0]['content']
        
            if item.get('start_time'):
                current_speaker=speaker_start_times[item['start_time']]
            elif item['type'] == 'punctuation':
                line = line+content
            
            if current_speaker != speaker:
                if speaker:
                    lines.append({'speaker':speaker, 'line':line, 'time':time})
                    speaker1.append(speaker)
                    time1.append(str(datetime.timedelta(seconds=int(round(float((time)))))))
                    text1.append(line)
                    
                    
                line=content
                speaker=current_speaker
                time=item['start_time']
                
                
            elif item['type'] != 'punctuation':
                line = line + ' ' + content
                
        lines.append({'speaker':speaker, 'line':line,'time':time})
        
        speaker1.append(speaker)
        text1.append(line)
        time1.append(str(datetime.timedelta(seconds=int(round(float((time)))))))
        sorted_lines = sorted(lines,key=lambda k: float(k['time']))
        for line_data in sorted_lines:
            #print(line_data)
            line='[' + str(datetime.timedelta(seconds=int(round(float(line_data['time']))))) + '] ' + line_data.get('speaker') + ': ' + line_data.get('line')
            #print(line)
    df1=pd.DataFrame(text1,columns=['text'])
    df1['speaker']=speaker1
    df1['time'] = time1
    df1=df1.drop(index=0,axis=0)
    #print(df1.drop(index=0,axis=0))
    return df1

In [20]:
data=speaker()
data

,text,speaker,time
1,"Biz yine bu konuda, iş sonuçlarıyla ilgili han...",spk_0,0:00:04
2,üretkenlik içinde şunu söyleyebilirim başlığı ...,spk_1,0:00:20
3,dakika da olsa,spk_2,0:01:46
4,sohbet etmek istiyor. Yirmi kişi bunu yapsa kı...,spk_1,0:01:47
5,ben Üretkenlikte Meclisi veya pozitif çok cidd...,spk_2,0:02:19
6,Peki ee daha yine en doğru ee bilgi aldığımız ...,spk_0,0:03:03


***Diyaloglar için DataFrame olarak alınabilecek kod (2. Yol);***

In [31]:
result = df.to_json(orient="split")
parsed = json.loads(result)

dictt=[]
for i in parsed['data']:
    dictt.append({
    'speaker':i[3],
    'start_time':i[1],
    'end_time':i[2],
    'comment':i[4]
    })
    
#dictt

In [32]:
def speaker2():
    
    index=0
    dicti=[]
    for i in dictt:
        temp={}
        try:
            if index==0:
                #print("index=0")
                dicti.append(dictt[index])

            elif dictt[index]['speaker']!=dictt[index-1]['speaker']:
                #print("bir önceki farklı ise speaker")
                dicti.append({
                        'speaker': dictt[index]['speaker'],
                        'start_time':dictt[index]['start_time'],
                        'end_time':dictt[index]['end_time'],
                        'comment':dictt[index]['comment']
                })

            else:
                #print("bir önceki ile aynı ise speaker")
                try:
                    newindex=len(dicti)-1
                    """
                    temp={
                        'speaker': dictt[index]['speaker'],
                        'start_time':dictt[index-1]['start_time'],
                        'end_time':dictt[index]['end_time'],
                        'comment':dictt[index-1]['comment']+'.'+dictt[index]['comment']
                    }
                    """
                    dicti[newindex]['speaker']=dictt[index]['speaker']
                    dicti[newindex]['start_time']=dicti[newindex]['start_time']
                    dicti[newindex]['end_time']=dictt[index]['end_time']
                    dicti[newindex]['comment']=dicti[newindex]['comment']+"."+dictt[index]['comment']
                
                except IndexError as e:
                    #print("hata verdi!")
                    dicti.append(temp)
        except IndexError as e:
            print(e)
        #print(index)
        index+=1
    a = pd.DataFrame.from_dict(dicti)
    return a

In [33]:
data = speaker2()
data

,speaker,start_time,end_time,comment
0,spk_0,00:00:04,00:00:14,"Biz yine bu konuda, iş sonuçlarıyla ilgili han..."
1,spk_1,00:00:20,00:01:46,üretkenlik içinde şunu söyleyebilirim başlığı ...
2,spk_2,00:01:46,00:01:46,dakika da olsa
3,spk_1,00:01:46,00:02:19,sohbet etmek istiyor. Yirmi kişi bunu yapsa kı...
4,spk_2,00:02:19,00:02:59,ben Üretkenlikte Meclisi veya pozitif çok cidd...
5,spk_0,00:03:02,00:03:09,Peki ee daha yine en doğru ee bilgi aldığımız ...


## Sentiment Analysis

In [24]:
## Turkish Bert Pipeline
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

model = AutoModelForSequenceClassification.from_pretrained("savasy/bert-base-turkish-sentiment-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-sentiment-cased")
sa= pipeline("sentiment-analysis", tokenizer=tokenizer, model=model)

In [25]:
sentiment = []
score = []
for i in data['comment']:
    sentiment.append(sa(i)[0]['label'])
    score.append(sa(i)[0]['score'])
data['sentiment']=sentiment
data['score']=score

In [26]:
data

,speaker,start_time,end_time,comment,sentiment,score
0,spk_0,00:00:04,00:00:14,"Biz yine bu konuda, iş sonuçlarıyla ilgili han...",negative,0.948679
1,spk_1,00:00:20,00:01:46,üretkenlik içinde şunu söyleyebilirim başlığı ...,positive,0.845118
2,spk_2,00:01:46,00:01:46,dakika da olsa,negative,0.960027
3,spk_1,00:01:46,00:02:19,sohbet etmek istiyor. Yirmi kişi bunu yapsa kı...,positive,0.955773
4,spk_2,00:02:19,00:02:59,ben Üretkenlikte Meclisi veya pozitif çok cidd...,negative,0.997754
5,spk_0,00:03:02,00:03:09,Peki ee daha yine en doğru ee bilgi aldığımız ...,positive,0.995301


***Konuşmacıların genel duygu durumları;***

In [27]:
speaker_score = data['sentiment'].groupby(data['speaker'])
speaker_score = pd.DataFrame(speaker_score.max())
speaker_score

,sentiment
speaker,
spk_0,positive
spk_1,positive
spk_2,negative


***Genel durum (Pozitif-Negatif-Notr) ;***

In [28]:
def pos_neg_notr():
    pos_val = 0
    neg_val = 0
    notr_val = 0
    
    for sent in data['sentiment']:
        if sent =='positive':
            pos_val += 1
            #print(pos_val)
        elif sent =='negative':
            neg_val += 1
            #print(neg_val)
        else:
            notr_val +1
    count = [pos_val, neg_val, notr_val]
    count = pd.DataFrame(list(count))
    count.index =['pos_val', 'neg_val', 'notr_val']
    count.columns = ['count']
    return count

In [29]:
pos_neg_notr = pos_neg_notr()
pos_neg_notr

,count
pos_val,3
neg_val,3
notr_val,0


In [34]:
data

,speaker,start_time,end_time,comment
0,spk_0,00:00:04,00:00:14,"Biz yine bu konuda, iş sonuçlarıyla ilgili han..."
1,spk_1,00:00:20,00:01:46,üretkenlik içinde şunu söyleyebilirim başlığı ...
2,spk_2,00:01:46,00:01:46,dakika da olsa
3,spk_1,00:01:46,00:02:19,sohbet etmek istiyor. Yirmi kişi bunu yapsa kı...
4,spk_2,00:02:19,00:02:59,ben Üretkenlikte Meclisi veya pozitif çok cidd...
5,spk_0,00:03:02,00:03:09,Peki ee daha yine en doğru ee bilgi aldığımız ...


In [43]:
a = data['comment'][0].split("?")
a

['Biz yine bu konuda, iş sonuçlarıyla ilgili hani bu rakamlar neler ifade ediyor',
 ' Yine sahaya sorduk ve Ee sahadan dinleyeceğiz. Biraz daha']